In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import os
import json
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/'멀티캠퍼스 5조'/2.데이터

df = pd.read_csv('통합데이터_표준화.csv')
df.head(3)

/content/drive/.shortcut-targets-by-id/1Mz49-WiynxJxHmmXYuWSqduV3-jd-BC7/멀티캠퍼스 5조/2.데이터


,출발지구군,출발지상세,예정일시,승차일시,시간대_30분,요일,월,화,수,목,금,토,일,언급량,강수량,적설량,평균속도,운행차량수,대기시간,대기시간_초
0,강북구,송천동,2019-01-01 05:20:00,2019-01-01 05:38:53,10,1,0,1,0,0,0,0,0,0.060216,0.0,0.0,0.876199,0.000000,18.0,1133.0
1,은평구,역촌동,2019-01-01 07:00:00,2019-01-01 07:35:42,14,1,0,1,0,0,0,0,0,0.527435,0.0,0.0,0.982563,0.275439,35.0,2142.0
2,도봉구,창제3동,2019-01-01 07:00:00,2019-01-01 07:30:04,14,1,0,1,0,0,0,0,0,0.590379,0.0,0.0,0.904969,0.275439,30.0,1804.0


# 다중선형회귀

In [ ]:
df.columns

Index(['출발지구군', '출발지상세', '예정일시', '승차일시', '시간대_30분', '요일', '월', '화', '수', '목',
       '금', '토', '일', '언급량', '강수량', '적설량', '평균속도', '운행차량수', '대기시간', '대기시간_초'],
      dtype='object')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# 배열로 만들기
X = np.array(df[['시간대_30분', '요일', '언급량', '월', '화', '수', '목',
       '금', '토', '일','강수량', '적설량', '평균속도', '운행차량수']])
y = np.array(df['대기시간'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,test_size= 0.1, random_state=2022
)

In [ ]:
res = ols('대기시간 ~ C(시간대_30분)', data = df).fit()
print(res.summary())

#  F - vlaue            : 회귀식의 존재 유무를 결정
#       H0 : 모든 회귀계수는 0이다.
#       H1 : 회귀계수 중 적어도 하나는 0이 아니다.
#       여기서 F-statistic이 0.05보다 작아야 되는데, 여기서는 크기 때문에 회귀식이 존재하지 않다고 봐야됩니다...ㅠ

                            OLS Regression Results                            
Dep. Variable:                   대기시간   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     5833.
Date:                Sun, 15 May 2022   Prob (F-statistic):               0.00
Time:                        14:53:34   Log-Likelihood:            -1.3257e+07
No. Observations:             2991602   AIC:                         2.651e+07
Df Residuals:                 2991554   BIC:                         2.651e+07
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           38.8197      0.717  

In [ ]:
from xgboost import XGBRegressor

xgr = XGBRegressor()
xgr.fit(X_train, y_train)
pred_xgr = xgr.predict(X_test)
r2_xgr= r2_score(y_test, pred_xgr)
mse_xgr = mean_squared_error(y_test, pred_xgr)

[14:54:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
r2_xgr, mse_xgr

(0.25122717798265215, 337.40115600135226)

### 시간대 변경
- ~6시 : 밤새벽
- 6시 ~ 10시 : 출근시간
- 10시 ~ 12시 : 점심시간
- 12시 ~ 15시 : 오후시간
- 15시 ~ 20시 : 퇴근시간
- 20 ~ : 밤새벽

In [ ]:
#lst = ['밤새벽', '출근시간', '점심시간', '오후시간', '퇴근시간']
add_lst = []

for i in tqdm(df.index):
    if df.loc[i, '시간대_30분'] <= 12 or df.loc[i, '시간대_30분'] >= 40:
        val = 0
    elif df.loc[i, '시간대_30분'] <= 20:
        val = 1
    elif df.loc[i, '시간대_30분'] <= 24:
        val = 2
    elif df.loc[i, '시간대_30분'] <= 30:
        val = 2
    else:
        val = 3
    add_lst.append(val)

100%|██████████| 2991602/2991602 [02:41<00:00, 18503.62it/s]


In [ ]:
df['시간대_30분'] = add_lst
df.head(10)

,출발지구군,출발지상세,예정일시,승차일시,시간대_30분,요일,월,화,수,목,금,토,일,언급량,강수량,적설량,평균속도,운행차량수,대기시간,대기시간_초
0,강북구,송천동,2019-01-01 05:20:00,2019-01-01 05:38:53,0,1,0,1,0,0,0,0,0,0.060216,0.0,0.0,0.876199,0.000000,18.0,1133.0
1,은평구,역촌동,2019-01-01 07:00:00,2019-01-01 07:35:42,1,1,0,1,0,0,0,0,0,0.527435,0.0,0.0,0.982563,0.275439,35.0,2142.0
2,도봉구,창제3동,2019-01-01 07:00:00,2019-01-01 07:30:04,1,1,0,1,0,0,0,0,0,0.590379,0.0,0.0,0.904969,0.275439,30.0,1804.0
3,도봉구,쌍문제4동,2019-01-01 07:00:00,2019-01-01 07:19:37,1,1,0,1,0,0,0,0,0,0.321479,0.0,0.0,0.904969,0.275439,19.0,1177.0
4,중구,신당제4동,2019-01-01 07:00:00,2019-01-01 07:14:49,1,1,0,1,0,0,0,0,0,0.248609,0.0,0.0,0.959895,0.275439,14.0,889.0
5,은평구,수색동,2019-01-01 07:00:00,2019-01-01 07:21:16,1,1,0,1,0,0,0,0,0,0.244464,0.0,0.0,0.982563,0.275439,21.0,1276.0
6,강북구,수유제3동,2019-01-01 07:00:00,2019-01-01 07:23:28,1,1,0,1,0,0,0,0,0,0.317443,0.0,0.0,0.801221,0.275439,23.0,1408.0
7,강북구,송중동,2019-01-01 07:34:00,2019-01-01 08:01:29,1,1,0,1,0,0,0,0,0,0.453147,0.0,0.0,0.801221,0.275439,27.0,1649.0
8,중구,신당제4동,2019-01-01 07:39:00,2019-01-01 08:10:02,1,1,0,1,0,0,0,0,0,0.154249,0.0,0.0,0.959895,0.275439,31.0,1862.0
9,강북구,번제3동,2019-01-01 07:49:00,2019-01-01 07:59:07,1,1,0,1,0,0,0,0,0,0.633904,0.0,0.0,0.801221,0.275439,10.0,607.0


In [ ]:
df.rename(columns={'시간대_30분':'시간대'}, inplace = True)

### 시간대 나눠서 XBG

In [ ]:
X = np.array(df[['시간대', '요일', '월', '화', '수', '목',
       '금', '토', '일', '언급량', '강수량', '적설량', '평균속도', '운행차량수']])
y = np.array(df['대기시간'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,test_size= 0.1, random_state=2022
)

In [ ]:
res = ols('대기시간 ~ C(시간대)*C(요일)', data = df).fit()
print(res.summary())

In [ ]:
xgr = XGBRegressor()
xgr.fit(X_train, y_train)
pred_xgr = xgr.predict(X_test)
r2_xgr= r2_score(y_test, pred_xgr)
mse_xgr = mean_squared_error(y_test, pred_xgr)

[15:25:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
r2_xgr, mse_xgr

(0.2047598865301502, 358.33957335748147)

# 년도 조정해보기 (2019~2020)

In [ ]:
df.loc[1, '예정일시'][:4]

'2019'

In [ ]:
add_lst = []
for i in tqdm(df.index):
    add_lst.append(df.loc[i, '예정일시'][:4])

100%|██████████| 2991602/2991602 [00:49<00:00, 60819.72it/s]


In [ ]:
df['year'] = add_lst
temp = df[(df.year != '2021')]

In [ ]:
X = np.array(temp[['시간대_30분', '요일', '월', '화', '수', '목',
       '금', '토', '일', '언급량', '강수량', '적설량', '평균속도', '운행차량수']])
y = np.array(temp['대기시간'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,test_size= 0.1, random_state=2022
)

In [ ]:
xgr = XGBRegressor()
xgr.fit(X_train, y_train)
pred_xgr = xgr.predict(X_test)
r2_xgr= r2_score(y_test, pred_xgr)
mse_xgr = mean_squared_error(y_test, pred_xgr)

[15:48:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
r2_xgr, mse_xgr

(0.26062157469800495, 359.180636359916)

In [ ]:
# train test 직접 분리
X_train = temp[temp.year != '2022'][['시간대_30분', '요일', '월', '화', '수', '목',
       '금', '토', '일', '언급량', '강수량', '적설량', '평균속도', '운행차량수']]
X_test = temp[temp.year == '2022'][['시간대_30분', '요일', '월', '화', '수', '목',
       '금', '토', '일', '언급량', '강수량', '적설량', '평균속도', '운행차량수']]
y_train = temp[temp.year != '2022']['대기시간']
y_test = temp[temp.year == '2022']['대기시간']

In [ ]:
xgr = XGBRegressor()
xgr.fit(X_train, y_train)
pred_xgr = xgr.predict(X_test)
r2_xgr= r2_score(y_test, pred_xgr)
mse_xgr = mean_squared_error(y_test, pred_xgr)

[15:52:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
r2_xgr, mse_xgr

(0.12869789568216095, 298.3376583884918)